In [48]:
import pandas as pd
import numpy as np

In [49]:
#Filter only EPL teams
df = pd.read_csv('players_data_light-2024_2025.csv')
pl_players = df[df['Comp'] == 'eng Premier League']

#Sort for same Comp and then Role of the players
sort = pl_players.sort_values(by=['Squad', 'Pos'])
sort.to_csv('players_data_light-2024_2025_copy.csv', index=False)

In [50]:
#There are 4 roles include GK, DF, MF, FW -> For each role we take different stats
#General stats for all players will include Age, Min/Match (= Min / MP)
#Stats for GK: Save%, GA/Match (= Goal Agaisnt/MP), CS% (Clean Sheet), PKsv% (Penalty saves)
#Stats for DF: Clr/Match (Clearences), TklW% (= TklW/Tkl), Blocks/Match, Int/Match, Err/Match
#Stats for MF: G+A/Match, xAG (Expected Assisted Goals), PrgC (Pogressive Carries), PrgP (Pogressive Passes), KP/Match(Key Passes)
#Stats for FW: G+A-PK/Match (Goals and Assist excluding Penalty Kick), npxG (Non Penalty Expected Goals), PrgR (Pogressive Run), SoT% (Shot on Target), Sh/90, 

In [51]:
#1. Calculating stats
sort.rename(columns={'Blocks_stats_defense': 'Blocks'}, inplace=True)
calculate_col = ['Min', 'GA', 'Clr', 'Blocks', 'Int', 'Err', 'G+A', 'G+A-PK', 'KP']
for col in calculate_col:
    sort[f'{col}/Match'] = sort[col] / sort['MP']
sort['PKsv%'] = np.where(sort['PKatt_stats_keeper'] > 0, (sort['PKsv'] / sort['PKatt_stats_keeper']) * 100, 0)
sort['TklW%'] = np.where(sort['Tkl'] > 0, (sort['TklW'] / sort['Tkl']) * 100, 0)

#2. Columns
# General stats
general_cols = ['Player', 'Squad', 'Pos', 'Age', 'Min/Match']

# Role stats
gk_stats = ['Save%', 'GA/Match', 'CS%', 'PKsv%']
df_stats = ['Clr/Match', 'TklW%', 'Blocks/Match', 'Int/Match', 'Err/Match']
mf_stats = ['G+A/Match', 'xAG', 'PrgC', 'PrgP', 'KP/Match']
fw_stats = ['G+A-PK/Match', 'npxG', 'PrgR', 'SoT%', 'Sh/90']

ordered_columns = general_cols + gk_stats + df_stats + mf_stats + fw_stats

#3. Assign NULL to other role that not Pos of the players
final_df = sort[ordered_columns].copy()
def assign_null(row):
    pos_list = row['Pos'].split(',')
    is_gk, is_df, is_mf, is_fw = 'GK' in pos_list, 'DF' in pos_list, 'MF' in pos_list, 'FW' in pos_list
    
    if not is_gk:
        for col in gk_stats: row[col] = np.nan
    if not is_df:
        for col in df_stats: row[col] = np.nan
    if not is_mf:
        for col in mf_stats:
            row[col] = np.nan
    if not is_fw:
        for col in fw_stats: row[col] = np.nan
    return row

final_df = final_df.apply(assign_null, axis=1)

#4. Round values to 2 decimal places
round_cols = final_df.select_dtypes(include=[np.number]).columns
final_df[round_cols] = final_df[round_cols].round(2)

In [52]:
#Save to file CSV
final_df.to_csv('players_stat.csv', index=False)

In [54]:
#Divide each team into a file CSV
squads = final_df['Squad'].unique()

for squad in squads:
    team_df = final_df[final_df['Squad'] == squad]
    
    # Create file
    filename = squad.replace(' ', '_') + '.csv'
    team_df.to_csv(filename, index=False)